In [13]:
# view datasets
import numpy as np
import pickle

with open('datasets/Neural_Source.pkl', 'rb') as f:
    train_data1 = pickle.load(f)['data']
train_trial_spikes1, train_trial_vel1 = train_data1['firing_rates'], train_data1['velocity']
a, b = 0, 0
for i in range(len(train_trial_spikes1)):
    print("i:", i)
    print(train_trial_spikes1[i].shape, train_trial_vel1[i].shape)

# with open('datasets/Neural_Target.pkl', 'rb') as f:
#     test_data = pickle.load(f)['data']
# test_trial_spikes, test_trial_vel = test_data['firing_rates'], test_data['velocity']
# for i in range(len(test_trial_spikes)):
#     print("i:", i)
#     print(test_trial_spikes[i].shape, test_trial_vel[i].shape)

i: 0
(59, 187) (59, 2)
i: 1
(58, 187) (58, 2)
i: 2
(50, 187) (50, 2)
i: 3
(57, 187) (57, 2)
i: 4
(45, 187) (45, 2)
i: 5
(46, 187) (46, 2)
i: 6
(43, 187) (43, 2)
i: 7
(44, 187) (44, 2)
i: 8
(48, 187) (48, 2)
i: 9
(51, 187) (51, 2)
i: 10
(47, 187) (47, 2)
i: 11
(46, 187) (46, 2)
i: 12
(53, 187) (53, 2)
i: 13
(47, 187) (47, 2)
i: 14
(44, 187) (44, 2)
i: 15
(45, 187) (45, 2)
i: 16
(57, 187) (57, 2)
i: 17
(55, 187) (55, 2)
i: 18
(53, 187) (53, 2)
i: 19
(53, 187) (53, 2)
i: 20
(49, 187) (49, 2)
i: 21
(50, 187) (50, 2)
i: 22
(45, 187) (45, 2)
i: 23
(43, 187) (43, 2)
i: 24
(46, 187) (46, 2)
i: 25
(47, 187) (47, 2)
i: 26
(45, 187) (45, 2)
i: 27
(45, 187) (45, 2)
i: 28
(47, 187) (47, 2)
i: 29
(51, 187) (51, 2)
i: 30
(50, 187) (50, 2)
i: 31
(47, 187) (47, 2)
i: 32
(44, 187) (44, 2)
i: 33
(65, 187) (65, 2)
i: 34
(46, 187) (46, 2)
i: 35
(45, 187) (45, 2)
i: 36
(45, 187) (45, 2)
i: 37
(48, 187) (48, 2)
i: 38
(45, 187) (45, 2)
i: 39
(48, 187) (48, 2)
i: 40
(51, 187) (51, 2)
i: 41
(46, 187) (46, 2)
i:

In [22]:
!.venv/bin/python3 VAE_Diffusion_CoTrain.py

(209, 37, 187)
(209, 37, 2)
10
/home/yusayura/courses/trust-proj/erdiff/ERDiff/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/home/yusayura/courses/trust-proj/erdiff/ERDiff/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/home/yusayura/courses/trust-proj/erdiff/ERDiff/VAE_Diffusion_CoTrain.py:185: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_no

In [ ]:
!python3 MLA.py

In [16]:

len_trial, num_neurons_s, num_neurons_t = 37, 187, 172

with open('datasets/Neural_Source.pkl', 'rb') as f:
    train_data1 = pickle.load(f)['data']

with open('datasets/Neural_Target.pkl', 'rb') as f:
    test_data = pickle.load(f)['data']

train_trial_spikes1, train_trial_vel1 = train_data1['firing_rates'], train_data1['velocity']

test_trial_spikes, test_trial_vel = test_data['firing_rates'], test_data['velocity']
# print(np.shape(train_trial_vel[0]))
start_pos = 1

train_trial_spikes_tide1 = np.array(
    [spike[start_pos:len_trial + start_pos, :num_neurons_s] for spike in train_trial_spikes1])
print(np.shape(train_trial_spikes_tide1))

train_trial_vel_tide1 = np.array([spike[start_pos:len_trial + start_pos, :] for spike in train_trial_vel1])
print(np.shape(train_trial_vel_tide1))

test_trial_spikes_tide = np.array([spike[:len_trial, :num_neurons_t] for spike in test_trial_spikes])
print(np.shape(test_trial_spikes_tide))

test_trial_vel_tide = np.array([spike[:len_trial, :] for spike in test_trial_vel])
print(np.shape(test_trial_vel_tide))

bin_width = float(0.02) * 1000

train_trial_spikes_tide = train_trial_spikes_tide1
train_trial_vel_tide = train_trial_vel_tide1

kern_sd_ms = 100
kern_sd = int(round(kern_sd_ms / bin_width))
import scipy.signal as signal
window = signal.gaussian(kern_sd, kern_sd, sym=True)
window /= np.sum(window)
filt = lambda x: np.convolve(x, window, 'same')

train_trial_spikes_smoothed = np.apply_along_axis(filt, 1, train_trial_spikes_tide)
test_trial_spikes_smoothed = np.apply_along_axis(filt, 1, test_trial_spikes_tide)

indices = np.arange(train_trial_spikes_tide.shape[0])
np.random.seed(2023)
np.random.shuffle(indices)
train_len = round(len(indices) * 0.8)
real_train_trial_spikes_smed, val_trial_spikes_smed = train_trial_spikes_smoothed[indices[:train_len]], \
    train_trial_spikes_smoothed[indices[train_len:]]
real_train_trial_vel_tide, val_trial_vel_tide = train_trial_vel_tide[indices[:train_len]], train_trial_vel_tide[
    indices[train_len:]]

real_train_trial_spikes_stand = (real_train_trial_spikes_smed)
val_trial_spikes_stand = (val_trial_spikes_smed)
test_trial_spikes_stand = (test_trial_spikes_smoothed)


(209, 37, 187)
(209, 37, 2)
(184, 37, 172)
(184, 37, 2)
